In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython import display
from climbing_wire.homography.homography import compute_homography
from climbing_wire.homography.homography import perspective_transform
from climbing_wire.landmark.compute import PoseImg
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.landmark.drawing import draw_landmarks
from climbing_wire.landmark.landmark_list import LandmarkListImg
from climbing_wire.utils.cv import cv_imshow
from climbing_wire.utils.data import get_package_fol
from climbing_wire.utils.mediapipe import JOINT_NAMES
from climbing_wire.utils.mediapipe import get_default_pose_connections
from climbing_wire.video.load import iterate_video_frames, pairwise_video_frames
from matplotlib import pyplot as plt
from pathlib import Path
from typing import List, Tuple, cast
import cv2 as cv
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.pose as mp_pose
import numpy as np
import pathlib
import sys


In [ ]:
in_fol = Path("/mnt/c/Users/nobilip/Videos/dataset/rock")
# in_fn = "PXL_20220901_202119640.mp4"
in_fn = "PXL_20230401_135540281.TS.mp4"

# in_fol = Path("~/data/rock")
# in_fn = "nonaka_miho_01.mp4"

in_vid_path = in_fol / in_fn


In [ ]:
import time
from tqdm import tqdm
from climbing_wire.joint_tracker.joint_tracker import JointTracker

pose_img_kwargs = {
    "static_image_mode": False,
    "model_complexity": 2,
    # "smooth_landmarks": True,
    # "enable_segmentation": False,
    # "smooth_segmentation": True,
    # "min_detection_confidence": 0.5,
    # "min_tracking_confidence": 0.5,
    "visibility_threshold": 0.05,
}

ax_params = dict(
    which="both",
    bottom=False,
    top=False,
    left=False,
    right=False,
    labelbottom=False,
    labeltop=False,
    labelleft=False,
    labelright=False,
)

jt = JointTracker(pose_img_kwargs=pose_img_kwargs)

jt_all: dict[int, JointTracker] = {}

max_frames = 12 * 4
msec_interval = 100

# pbar = tqdm(total=max_frames)
fig, axes = plt.subplots(4, 12, figsize=(20, 8))
axf = axes.flat

for frame_num, (frame1, frame2) in enumerate(
    pairwise_video_frames(in_vid_path, msec_interval=msec_interval)
):
    display.clear_output(wait=True)

    jt.process_frame_pair(frame1, frame2)

    # pbar.update(1)
    ann_img = frame2.copy()
    if jt.landlist is not None:
        draw_landmarks(ann_img, jt.landlist)
    ax = axf[frame_num]
    ax.tick_params(**ax_params)
    cv_imshow(ann_img, ax=ax)
    fig.tight_layout()
    display.display(fig)
    # display.clear_output(wait=True)

    start_time = time.time()
    jt_all[frame_num] = jt.copy()
    print(f"frame_num: {frame_num}, time: {time.time()-start_time:.2f}")

    # quit early if we've reached the max number of frames
    if frame_num >= max_frames:
        break

# pbar.close()
jt.close()


In [ ]:
track = jt_all[1].joint_hists["left_hand"].track
track


In [ ]:
# jt = jt_all[1]
# jt_some = list(jt_all.values())[-7:-6]
jt_some = list(jt_all.values())[-1:]
for jt in jt_some:
    fig, ax = plt.subplots(figsize=(6, 10))

    ann_img = jt.frame2.copy()
    # ann_img = np.zeros_like(jt.frame2)

    if jt.landlist is not None:
        print(jt.landlist)
        draw_landmarks(ann_img, jt.landlist)

    cv_imshow(ann_img, ax=ax)
    # ax.imshow(ann_img)

    for joint_name in JOINT_NAMES:
        track = jt.joint_hists[joint_name].track
        ax.scatter(*track.T, s=0.3)
        ax.plot(*track.T)

    ax.tick_params(**ax_params)
    plt.show()
    plt.close(fig)
